In [4]:
!pip install pandas pyxlsb

In [5]:
import pandas as pd
import numpy as np

In [6]:
# ============================================================
# PRUEBA TÉCNICA — CIENTÍFICO DE DATOS (SISTECREDITO)
# MODELO DE RIESGO TEMU — VALIDACIÓN TEMPORAL ESTRICTA (SIN FUGA)
# ============================================================
# Requiere: pandas, numpy, scikit-learn>=1.1, joblib, pyxlsb
# Opcional (explicabilidad): shap
# ------------------------------------------------------------
# Entradas esperadas (hoja Excel .xlsb):
#   - 'DataFramePrueba' con columnas (incluye target 'PerdidaCartera')
#   - 'diccionario' (descripciones de variables)
# ============================================================

import warnings, os, sys, json
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path
from pandas.api.types import is_numeric_dtype

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (
    roc_auc_score, average_precision_score, brier_score_loss,
    classification_report, confusion_matrix, precision_recall_curve
)
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.inspection import permutation_importance

import joblib

# ============================================================
# 0) CONFIGURACIÓN GENERAL
# ============================================================
FILE_PATH  = "/Users/karenaraque/Desktop/practica_cartera_temu/data/DataFramePrueba 2025_08.xlsb"
SHEET_DATA = "DataFramePrueba"
SHEET_DICT = "diccionario"

TARGET = "PerdidaCartera"    # 0/1
RANDOM_STATE = 42
TOP_K_LIST = [1, 2, 5, 10, 20]
GOAL_PRECISION = 0.80  # objetivo negocio (precision en morosos)

# Costeo opcional para EV (puedes editar). Unidades monetarias del negocio.
COST_FP   = 1.0   # costo de intervenir a un no-moroso
COST_FN   = 5.0   # pérdida si NO intervienes a un moroso
BENEFIT_TP= 4.0   # beneficio por intervenir a un moroso (recuperación)
COST_TN   = 0.0   # usualmente 0

ARTIF_DIR = Path("./artifacts_modelo"); ARTIF_DIR.mkdir(parents=True, exist_ok=True)
np.set_printoptions(precision=3, suppress=True)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)

# ============================================================
# 1) INGESTA Y SANITY CHECKS
# ============================================================
print(">>> Cargando Excel .xlsb ...")
try:
    excel_hojas = pd.ExcelFile(FILE_PATH, engine="pyxlsb")
    print("Hojas encontradas:", excel_hojas.sheet_names)
except Exception as e:
    raise RuntimeError(f"No pude abrir el archivo .xlsb en {FILE_PATH}: {e}")

clientes = pd.read_excel(FILE_PATH, sheet_name=SHEET_DATA, engine="pyxlsb")
try:
    diccionario = pd.read_excel(FILE_PATH, sheet_name=SHEET_DICT, engine="pyxlsb")
except Exception:
    diccionario = None
    print("Aviso: hoja 'diccionario' no disponible, continuo sin ella.")

print("\n>>> SHAPE crudo:", clientes.shape)
print(">>> Columnas (primeras 30):", list(clientes.columns)[:30])

# ============================================================
# 2) SANITY CHECKS de fechas (solo reporta, no transforma aún)
# ============================================================
def parse_dates_raw(df):
    out = df.copy()
    # FechaEvento se asume ISO-like (string). Intentamos parsear con UTC seguro.
    out['FechaEvento_dt'] = pd.to_datetime(out['FechaEvento'], errors='coerce', utc=True).dt.tz_convert(None)

    # Excel serial dates para otras fechas:
    out['FechaVinculacionCliente_dt'] = pd.to_datetime(out['FechaVinculacionCliente'], errors='coerce',
                                                       origin='1899-12-30', unit='D')
    out['FechaUltimoUso_dt_raw']      = pd.to_datetime(out['FechaUltimoUso'], errors='coerce',
                                                       origin='1899-12-30', unit='D')
    # PrimerUso a veces viene en 1904-epoch en algunos exportes
    fpu = pd.to_datetime(out['FechaPrimerUso'], errors='coerce', origin='1899-12-30', unit='D')
    fpu_alt = pd.to_datetime(out['FechaPrimerUso'], errors='coerce', origin='1904-01-01', unit='D')
    # elegimos el que tenga más fechas razonables (no NaT y no < 1900)
    valid_a = fpu.notna() & (fpu.dt.year >= 1900)
    valid_b = fpu_alt.notna() & (fpu_alt.dt.year >= 1900)
    out['FechaPrimerUso_dt_raw'] = np.where(valid_a | (~valid_b),
                                            fpu, fpu_alt)
    out['FechaPrimerUso_dt_raw'] = pd.to_datetime(out['FechaPrimerUso_dt_raw'], errors='coerce')
    return out

def report_date_anomalies(df):
    print("\n>>> Chequeos de integridad (fechas):")
    n = len(df)
    c1 = (df['FechaPrimerUso_dt_raw'] < df['FechaVinculacionCliente_dt']).sum()
    c2 = (df['FechaUltimoUso_dt_raw'] > df['FechaEvento_dt']).sum()
    c3 = df['FechaPrimerUso_dt_raw'].isna().sum()
    c4 = df['FechaUltimoUso_dt_raw'].isna().sum()
    print(f" - PrimerUso < Vinculacion: {c1} ({c1/n:.1%})")
    print(f" - UltimoUso  > Evento    : {c2} ({c2/n:.1%})")
    print(f" - PrimerUso NaT           : {c3} ({c3/n:.1%})")
    print(f" - UltimoUso  NaT          : {c4} ({c4/n:.1%})")
    # Lugares/fechas extremas para inspección
    if c1>0:
        ej = df.loc[df['FechaPrimerUso_dt_raw'] < df['FechaVinculacionCliente_dt'],
                    ['IdentificadorCliente','FechaPrimerUso_dt_raw','FechaVinculacionCliente_dt']].head(5)
        print("\n  Ejemplos PrimerUso<Vinculacion:\n", ej.to_string(index=False))
    if c2>0:
        ej2 = df.loc[df['FechaUltimoUso_dt_raw'] > df['FechaEvento_dt'],
                     ['IdentificadorCliente','FechaUltimoUso_dt_raw','FechaEvento_dt']].head(5)
        print("\n  Ejemplos UltimoUso>Evento:\n", ej2.to_string(index=False))

clientes = parse_dates_raw(clientes)
report_date_anomalies(clientes)

# ============================================================
# 3) FEATURE ENGINEERING (con variantes de tratamiento de fechas)
# ============================================================
EXCLUDE_FROM_FEATURES = {
    'IdentificadorCliente','DiasMora',
    'FechaEvento','FechaVinculacionCliente','FechaUltimoUso','FechaPrimerUso',
    'CodigoAlmacenEntregaTC','CodigoMunicipioEntregaTC'
}
CAT_CANDIDATES = ['CategoriaPrincipalCredito','UsoAppWeb','Genero',
                  'TipoMunicipioEntregaTC','CanalMunicipioEntregaTC']

def parse_and_features(df_raw: pd.DataFrame,
                       primeruso_strategy: str = "clip_a_vinc",
                       ultimo_uso_clip_evento: bool = True) -> pd.DataFrame:
    """
    primeruso_strategy:
      - 'clip_a_vinc': si PrimerUso< Vinc => usar Vinculacion (con flag)
      - 'poner_na'   : si PrimerUso< Vinc => NaT (con flag)
      - 'mantener'   : mantener valor (pero flag de anomalia)
    ultimo_uso_clip_evento:
      - True: si UltimoUso>Evento => clipear a Evento (con flag)
    """
    df = df_raw.copy()

    # Fechas base (ya parseadas arriba)
    df['FechaEvento_dt'] = df['FechaEvento_dt']
    df['FechaVinculacionCliente_dt'] = df['FechaVinculacionCliente_dt']

    # ---- Primer Uso
    df['Flag_PrimerUsoAntesVinc'] = (df['FechaPrimerUso_dt_raw'] < df['FechaVinculacionCliente_dt']).astype(int)
    if primeruso_strategy == "clip_a_vinc":
        df['FechaPrimerUso_dt'] = df['FechaPrimerUso_dt_raw']
        mask_bad = df['FechaPrimerUso_dt_raw'].isna() | (df['FechaPrimerUso_dt_raw'] < df['FechaVinculacionCliente_dt'])
        df.loc[mask_bad, 'FechaPrimerUso_dt'] = df.loc[mask_bad, 'FechaVinculacionCliente_dt']
    elif primeruso_strategy == "poner_na":
        df['FechaPrimerUso_dt'] = df['FechaPrimerUso_dt_raw']
        mask_bad = df['FechaPrimerUso_dt_raw'] < df['FechaVinculacionCliente_dt']
        df.loc[mask_bad, 'FechaPrimerUso_dt'] = pd.NaT
    elif primeruso_strategy == "mantener":
        df['FechaPrimerUso_dt'] = df['FechaPrimerUso_dt_raw']
    else:
        raise ValueError("primeruso_strategy inválida")

    # ---- Último Uso
    df['Flag_UltimoUsoPosteriorEvento'] = (df['FechaUltimoUso_dt_raw'] > df['FechaEvento_dt']).astype(int)
    if ultimo_uso_clip_evento:
        df['FechaUltimoUso_dt'] = df['FechaUltimoUso_dt_raw'].copy()
        mask_bad2 = df['FechaUltimoUso_dt_raw'] > df['FechaEvento_dt']
        df.loc[mask_bad2, 'FechaUltimoUso_dt'] = df.loc[mask_bad2, 'FechaEvento_dt']
    else:
        df['FechaUltimoUso_dt'] = df['FechaUltimoUso_dt_raw']

    # ---- Diferencias temporales seguras
    def safe_months(a, b):
        d = (a - b).dt.days
        d = d.where(d.notna(), 0); d = np.where(d < 0, 0, d)
        return d / 30.0

    def safe_days(a, b):
        d = (a - b).dt.days
        d = d.where(d.notna(), 0); d = np.where(d < 0, 0, d)
        return d

    df['MesesDesdeVinculacion'] = safe_months(df['FechaEvento_dt'], df['FechaVinculacionCliente_dt'])
    df['MesesDesdePrimerUso']   = safe_months(df['FechaEvento_dt'], df['FechaPrimerUso_dt'])
    df['DiasDesdeUltimoUso']    = safe_days(df['FechaEvento_dt'],  df['FechaUltimoUso_dt'])

    # ---- Numéricas clave + flags
    keep_nums = [
        'UsabilidadCupo','DiasMaximosMoraCreditosGenerados','NumeroCreditosGPrevius',
        'NumeroCreditosGCanalFPrevius','NumeroCreditosGEstadoActivosPrevius','NumeroCreditosGEstadoPagadosPrevius',
        'NumeroCreditosGCanalVPrevius','NumeroCreditosLPrevius','NumeroCreditosLEstadoActivosPrevius',
        'NumeroCreditosLEstadoPagadosPrevius','TotalPagosEfectuadosGlobalmentePrevius','TotalPagosEfectuadosLocalmentePrevius',
        'NumeroIntentosFallidos','CupoAprobado','ScoreCrediticio','Edad',
        'MesesDesdeVinculacion','MesesDesdePrimerUso','DiasDesdeUltimoUso'
    ]
    for c in keep_nums:
        if c in df.columns:
            df[f'Flag_{c}_NaN'] = df[c].isna().astype(int)

    # UsabilidadCupo
    df['UsabilidadCupo'] = pd.to_numeric(df.get('UsabilidadCupo', np.nan), errors='coerce')
    df['Flag_Usab_Outlier']= ((df['UsabilidadCupo'] < 0) | (df['UsabilidadCupo'] > 2)).astype(int)
    df.loc[(df['UsabilidadCupo'] < 0) | (df['UsabilidadCupo'] > 2), 'UsabilidadCupo'] = np.nan
    df['UsabilidadCupo'] = df['UsabilidadCupo'].fillna(df['UsabilidadCupo'].median())

    # Score & Cupo
    df['ScoreSinInfo'] = (df['ScoreCrediticio'].fillna(0) == 0).astype(int)
    df['ScoreCrediticio'] = df['ScoreCrediticio'].fillna(df['ScoreCrediticio'][df['ScoreCrediticio']>0].median())
    df.loc[df['ScoreCrediticio'] < 0, 'ScoreCrediticio'] = 0
    df['log_CupoAprobado'] = np.log1p(df['CupoAprobado'].fillna(df['CupoAprobado'].median()))

    # Categóricas limpias
    for c in CAT_CANDIDATES:
        if c in df.columns:
            df[c] = df[c].fillna('Desconocido')
    if 'Genero' in df.columns:
        df['Genero'] = df['Genero'].replace({27:'Desconocido'})

    if 'TipoMunicipioEntregaTC' in df.columns:
        df['TipoMunicipioEntregaTC'] = df['TipoMunicipioEntregaTC'].replace({'PEQUEÃ‘O':'PEQUEÑO'}).fillna('Desconocido')

    # Derivadas
    df['Flag_PrimerUsoTemu'] = (df['NumeroCreditosGPrevius'].fillna(0) == 0).astype(int)
    df['ratio_pagos_local_global'] = (
        df['TotalPagosEfectuadosLocalmentePrevius'].fillna(0) /
        (df['TotalPagosEfectuadosGlobalmentePrevius'].fillna(0) + 1.0)
    ).clip(0,1)

    df['creditos_activos_ratio'] = (
        df['NumeroCreditosGEstadoActivosPrevius'].fillna(0) /
        (df['NumeroCreditosGPrevius'].fillna(0) + 1.0)
    ).clip(0,1)

    # Buckets score
    df['ScoreBucket'] = 'sin_info'
    mask_pos = df['ScoreCrediticio'] > 0
    if mask_pos.sum() > 0:
        q1, q2 = df.loc[mask_pos, 'ScoreCrediticio'].quantile([0.33, 0.66]).values
        df.loc[mask_pos & (df['ScoreCrediticio'] <= q1), 'ScoreBucket'] = 'bajo'
        df.loc[mask_pos & (df['ScoreCrediticio'] >  q1) & (df['ScoreCrediticio'] <= q2), 'ScoreBucket'] = 'medio'
        df.loc[mask_pos & (df['ScoreCrediticio'] >  q2), 'ScoreBucket'] = 'alto'
    df['ScoreBucket'] = pd.Categorical(df['ScoreBucket'], categories=['sin_info','bajo','medio','alto'], ordered=True)

    # Winsorización (p99) + flags
    def cap_with_flag(s, upper):
        s = s.fillna(0)
        flag = (s > upper).astype(int)
        return np.where(s > upper, upper, s), flag

    cap_cols = [
        'DiasDesdeUltimoUso','MesesDesdeVinculacion',
        'TotalPagosEfectuadosGlobalmentePrevius','TotalPagosEfectuadosLocalmentePrevius',
        'NumeroCreditosGPrevius','NumeroCreditosGCanalFPrevius','NumeroCreditosGCanalVPrevius',
        'NumeroCreditosGEstadoActivosPrevius','NumeroCreditosGEstadoPagadosPrevius',
        'NumeroCreditosLPrevius','NumeroCreditosLEstadoActivosPrevius','NumeroCreditosLEstadoPagadosPrevius'
    ]
    for c in cap_cols:
        if c in df.columns:
            p99 = df[c].quantile(0.99)
            capped, flag = cap_with_flag(df[c], p99)
            df[c] = capped
            df[f'Flag_{c}_Capped'] = flag

    return df


def build_feature_lists(df: pd.DataFrame):
    drop = set(EXCLUDE_FROM_FEATURES) | {
        'FechaEvento_dt','FechaVinculacionCliente_dt','FechaUltimoUso_dt_raw','FechaPrimerUso_dt_raw'
    }
    num_base = [
        'UsabilidadCupo','MesesDesdeVinculacion','MesesDesdePrimerUso','DiasDesdeUltimoUso',
        'NumeroCreditosGPrevius','NumeroCreditosGCanalFPrevius','NumeroCreditosGEstadoActivosPrevius',
        'NumeroCreditosGEstadoPagadosPrevius','NumeroCreditosGCanalVPrevius',
        'NumeroCreditosLPrevius','NumeroCreditosLEstadoActivosPrevius','NumeroCreditosLEstadoPagadosPrevius',
        'TotalPagosEfectuadosGlobalmentePrevius','TotalPagosEfectuadosLocalmentePrevius',
        'NumeroIntentosFallidos','ScoreCrediticio','ScoreSinInfo','CupoAprobado','log_CupoAprobado','Edad',
        'Flag_PrimerUsoAntesVinc','Flag_Usab_Outlier','ratio_pagos_local_global','creditos_activos_ratio',
        'Flag_UltimoUsoPosteriorEvento'
    ]
    num_dyn = [c for c in df.columns if c.startswith('Flag_') and (c.endswith('_NaN') or c.endswith('_Capped'))]
    num_final = [c for c in (num_base + num_dyn) if c in df.columns and c not in drop]
    cat_final = [c for c in (CAT_CANDIDATES + ['Flag_PrimerUsoTemu','ScoreBucket']) if c in df.columns and c not in drop]
    overlap = set(num_final) & set(cat_final)
    if overlap:
        print(">>> Aviso: había columnas en num y cat. Se quitan de num:", sorted(list(overlap)))
        num_final = [c for c in num_final if c not in overlap]
    num_final = list(dict.fromkeys(num_final))
    cat_final = list(dict.fromkeys(cat_final))
    return num_final, cat_final


def prune_low_variance(X: pd.DataFrame):
    low_var = [c for c in X.columns if X[c].nunique(dropna=False) <= 1]
    if low_var:
        print(">>> Poda de columnas casi-constantes:", low_var)
        X = X.drop(columns=low_var, errors='ignore')
    else:
        print(">>> Poda de columnas casi-constantes: ninguna")
    return X, low_var if low_var else []


def build_model(cat_cols, num_cols):
    pre = ColumnTransformer(
        transformers=[
            ('cat', Pipeline(steps=[
                ('imp', SimpleImputer(strategy='most_frequent')),
                ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
            ]), cat_cols),
            ('num', Pipeline(steps=[
                ('imp', SimpleImputer(strategy='median'))
            ]), num_cols)
        ],
        remainder='drop'
    )
    clf = HistGradientBoostingClassifier(
        learning_rate=0.07, max_leaf_nodes=31, l2_regularization=1.0,
        random_state=RANDOM_STATE
    )
    pipe = Pipeline([('pre', pre), ('clf', clf)])
    return pipe

def evaluate_proba(y_true, y_proba, name="model"):
    auc = roc_auc_score(y_true, y_proba)
    ap  = average_precision_score(y_true, y_proba)
    br  = brier_score_loss(y_true, y_proba)
    p, r, thr = precision_recall_curve(y_true, y_proba)
    f1 = 2*p*r/(p+r+1e-12)
    j  = np.argmax(f1)
    best_thr = thr[j-1] if j>0 and j-1 < len(thr) else 0.5
    y_hat = (y_proba >= best_thr).astype(int)

    print(f"[{name}] ROC-AUC={auc:.3f} | PR-AUC={ap:.3f} | Brier={br:.3f}")
    print(f"[{name}] Base rate y=1: {y_true.mean():.3f} | mean(p1)={np.mean(y_proba):.3f}")
    print(f"[{name}] best-F1={f1[j]:.3f} @ thr={best_thr:.3f}")
    print(f"[{name}] ConfMatrix @thr={best_thr:.3f}:\n", confusion_matrix(y_true, y_hat))
    print(f"[{name}] Report @thr={best_thr:.3f}:\n", classification_report(y_true, y_hat, digits=3))
    return dict(roc_auc=auc, pr_auc=ap, brier=br, thr=best_thr, p=p, r=r, thr_curve=thr)

def sweep_thresholds(y_true, y_proba, thresholds):
    rows = []
    for t in thresholds:
        yhat = (y_proba >= t).astype(int)
        TN, FP, FN, TP = confusion_matrix(y_true, yhat).ravel()
        prec1 = TP/(TP+FP) if TP+FP>0 else 0.0
        rec1  = TP/(TP+FN) if TP+FN>0 else 0.0
        rows.append({"thr": round(t,3),
                     "prec_1": round(prec1,3), "rec_1": round(rec1,3),
                     "TP": TP, "FP": FP, "FN": FN, "TN": TN})
    tab = pd.DataFrame(rows)
    print("\n[BARRIDO UMBRALES] (clase=1) primeras filas:\n", tab.head(10).to_string(index=False))
    return tab

def topk_gain_table(y_true, y_proba, top_k_list):
    n = len(y_true)
    order = np.argsort(-y_proba)
    y_sorted = np.array(y_true)[order]
    out = []
    base_rate = y_true.mean()
    cum_ones = np.cumsum(y_sorted)
    for k in top_k_list:
        m = max(1, int(n * k / 100.0))
        tp_k = int(cum_ones[m-1])
        rate_k = tp_k / m
        lift_k = rate_k / base_rate if base_rate>0 else np.nan
        out.append({"top_%": k, "n_alertas": m, "morosos_detectados": tp_k,
                    "tasa_moros_topk": round(rate_k,3), "lift_vs_base": round(lift_k,2)})
    tab = pd.DataFrame(out)
    print("\n[GANANCIA/LIFT por top-k%]\n", tab.to_string(index=False))
    return tab

def expected_value_at_threshold(y_true, y_proba, thr,
                                c_fp=COST_FP, c_fn=COST_FN, b_tp=BENEFIT_TP, c_tn=COST_TN):
    yhat = (y_proba >= thr).astype(int)
    TN, FP, FN, TP = confusion_matrix(y_true, yhat).ravel()
    EV = TP*b_tp - FP*c_fp - FN*c_fn - TN*c_tn
    return EV, TP, FP, FN, TN

def search_best_ev(y_true, y_proba, c_fp=COST_FP, c_fn=COST_FN, b_tp=BENEFIT_TP, c_tn=COST_TN):
    grid_thr = np.unique(np.round(np.linspace(0.01, 0.99, 99), 3))
    rows = []
    for t in grid_thr:
        ev, TP, FP, FN, TN = expected_value_at_threshold(y_true, y_proba, t, c_fp, c_fn, b_tp, c_tn)
        rows.append({"thr": t, "EV": ev, "TP":TP,"FP":FP,"FN":FN,"TN":TN})
    df = pd.DataFrame(rows).sort_values("EV", ascending=False)
    best = df.iloc[0].to_dict()
    print(f"\n[EV] Mejor umbral por valor económico: thr={best['thr']:.3f} | EV={best['EV']:.1f} | TP={best['TP']} FP={best['FP']} FN={best['FN']} TN={best['TN']}")
    return df, best

# ============================================================
# 4) SPLIT TEMPORAL 64/16/20 (train_in / valid_in / holdout)
# ============================================================
cut1 = clientes['FechaEvento_dt'].quantile(0.64)  # 64%
cut2 = clientes['FechaEvento_dt'].quantile(0.80)  # 64%+16% = 80%

train_in_idx = clientes['FechaEvento_dt'] <= cut1
valid_in_idx = (clientes['FechaEvento_dt'] > cut1) & (clientes['FechaEvento_dt'] <= cut2)
holdout_idx  = clientes['FechaEvento_dt'] > cut2

print("\n>>> Splits temporales:")
print(f"  train_in = {train_in_idx.sum()} filas (<= {pd.Timestamp(cut1).date()})")
print(f"  valid_in = {valid_in_idx.sum()} filas (<= {pd.Timestamp(cut2).date()})")
print(f"  holdout  = {holdout_idx.sum()} filas (>  {pd.Timestamp(cut2).date()})")

# ============================================================
# 5) EXPERIMENTOS (al estilo Chip Huyen) — ablation de fechas
# ============================================================
EXPERIMENTS = [
    {"name": "clip_a_vinc__clip_ultimo", "primeruso":"clip_a_vinc", "clip_ultimo": True},
    {"name": "poner_na__clip_ultimo",     "primeruso":"poner_na",    "clip_ultimo": True},
    {"name": "mantener__clip_ultimo",     "primeruso":"mantener",    "clip_ultimo": True},
]

exp_results = []

for exp in EXPERIMENTS:
    print("\n======================")
    print(f">>> EXPERIMENTO: {exp['name']} (PrimerUso={exp['primeruso']}, UltimoUsoClipEvento={exp['clip_ultimo']})")
    df_feat = parse_and_features(clientes, primeruso_strategy=exp['primeruso'],
                                 ultimo_uso_clip_evento=exp['clip_ultimo'])
    num_final, cat_final = build_feature_lists(df_feat)

    X = df_feat[num_final + cat_final].copy()
    y = df_feat[TARGET].astype(int).copy()

    # Poda
    X, dropped_lowvar = prune_low_variance(X)

    # Split
    X_tr_in, y_tr_in = X.loc[train_in_idx], y.loc[train_in_idx]
    X_va_in, y_va_in = X.loc[valid_in_idx], y.loc[valid_in_idx]
    X_ho,    y_ho    = X.loc[holdout_idx],  y.loc[holdout_idx]

    # Modelo y calibración SOLO en train_in + valid_in
    pipe = build_model(cat_final, [c for c in X.columns if c not in cat_final])
    sw = compute_sample_weight("balanced", y_tr_in)
    pipe.fit(X_tr_in, y_tr_in, clf__sample_weight=sw)

    cal = CalibratedClassifierCV(pipe, method='isotonic', cv='prefit')
    cal.fit(X_va_in, y_va_in)  # calibración estricta en VALID

    # Métricas en valid_in (para seleccionar variante)
    proba_va = cal.predict_proba(X_va_in)[:,1]
    mv = evaluate_proba(y_va_in, proba_va, name=f"VALID[{exp['name']}]")
    # Métricas en train_in (diagnóstico de sobreajuste)
    proba_tr = cal.predict_proba(X_tr_in)[:,1]
    mt = evaluate_proba(y_tr_in, proba_tr, name=f"TRAIN[{exp['name']}]")

    exp_results.append({
        "exp": exp["name"], "primeruso":exp["primeruso"], "clip_ultimo":exp["clip_ultimo"],
        "valid_pr_auc": mv["pr_auc"], "valid_roc_auc": mv["roc_auc"], "valid_brier": mv["brier"],
        "train_pr_auc": mt["pr_auc"], "train_roc_auc": mt["roc_auc"]
    })

# Comparativa estilo Chip Huyen
exp_df = pd.DataFrame(exp_results).sort_values(["valid_pr_auc","valid_roc_auc"], ascending=False)
print("\n====================== RESULTADOS DE EXPERIMENTOS (ordenado por PR-AUC VALID) ======================")
print(exp_df.to_string(index=False))

best = exp_df.iloc[0].to_dict()
print(f"\n>>> Variante ganadora por VALID: {best['exp']} (PR-AUC={best['valid_pr_auc']:.3f} | ROC-AUC={best['valid_roc_auc']:.3f})")

# ============================================================
# 6) ENTRENAR DEFINITIVO (train_in -> fit, valid_in -> calibrar) y EVALUAR en HOLDOUT
# ============================================================
# Reconstruimos con la mejor variante
best_exp = [e for e in EXPERIMENTS if e["name"] == best["exp"]][0]
df_best = parse_and_features(clientes, primeruso_strategy=best_exp['primeruso'],
                             ultimo_uso_clip_evento=best_exp['clip_ultimo'])
num_final, cat_final = build_feature_lists(df_best)
Xall = df_best[num_final + cat_final].copy()
yall = df_best[TARGET].astype(int).copy()
Xall, _ = prune_low_variance(Xall)

X_tr_in, y_tr_in = Xall.loc[train_in_idx], yall.loc[train_in_idx]
X_va_in, y_va_in = Xall.loc[valid_in_idx], yall.loc[valid_in_idx]
X_ho,    y_ho    = Xall.loc[holdout_idx],  yall.loc[holdout_idx]

print("\n>>> Entrenando ganador en train_in y calibrando en valid_in; evaluando en holdout (nunca visto).")
pipe = build_model(cat_final, [c for c in Xall.columns if c not in cat_final])
sw = compute_sample_weight("balanced", y_tr_in)
pipe.fit(X_tr_in, y_tr_in, clf__sample_weight=sw)

cal = CalibratedClassifierCV(pipe, method='isotonic', cv='prefit')
cal.fit(X_va_in, y_va_in)

# EVALUACIÓN FINAL
proba_ho = cal.predict_proba(X_ho)[:,1]
m = evaluate_proba(y_ho, proba_ho, name="HOLDOUT (final)")

# Reportes de negocio
_ = topk_gain_table(y_ho, proba_ho, TOP_K_LIST)

# Barrido y objetivo de precisión
grid_thr = np.unique(np.round(np.linspace(0.01, 0.99, 99), 3))
tab_thr = sweep_thresholds(y_ho, proba_ho, grid_thr)
tab_thr.to_csv(ARTIF_DIR/"holdout_threshold_sweep.csv", index=False)

p, r, thr = m["p"], m["r"], m["thr_curve"]
idx = np.where(p >= GOAL_PRECISION)[0]
if len(idx) == 0:
    print(f"\n>>> No hay punto con precision ≥ {GOAL_PRECISION:.2f} en holdout.")
else:
    k = idx[np.argmax(r[idx])]
    thr_goal = thr[k-1] if k>0 else 0.5
    yhat = (proba_ho >= thr_goal).astype(int)
    TN, FP, FN, TP = confusion_matrix(y_ho, yhat).ravel()
    prec1, rec1 = (TP/(TP+FP) if TP+FP>0 else 0), (TP/(TP+FN) if TP+FN>0 else 0)
    print(f"\n[Objetivo negocio] PRECISIÓN≥{GOAL_PRECISION:.2f} @thr={thr_goal:.3f}")
    print(f"  TP={TP} FP={FP} FN={FN} TN={TN}")
    print(f"  precision1={prec1:.3f} recall1={rec1:.3f}")
    print(f"  %alertados={(TP+FP)/len(y_ho):.3f}   %morosos_detectados={TP/len(y_ho):.3f}")

# Umbral por Máx-F1 (referencia)
j  = np.argmax(2*m["p"]*m["r"]/(m["p"]+m["r"]+1e-12))
thr_star = m["thr_curve"][j-1] if j>0 else 0.5
print(f"\n[Referencia] Máx-F1 @thr≈{thr_star:.3f} | precision={m['p'][j]:.3f} | recall={m['r'][j]:.3f}")

# Calibración por deciles
def decile_calibration(y_true, y_proba, name="holdout"):
    bins = pd.qcut(y_proba, q=10, duplicates='drop')
    tab = pd.DataFrame({"p":y_proba, "y":y_true}).groupby(bins, observed=True).agg(
        p_mean=('p','mean'),
        y_rate=('y','mean'),
        n=('p','size')
    ).reset_index()
    print(f"\n[{name}] Calibración por deciles (p_mean ~ y_rate):\n", tab.to_string(index=False))
    tab.to_csv(ARTIF_DIR/f"calibracion_deciles_{name}.csv", index=False)
    return tab
_ = decile_calibration(y_ho, proba_ho, name="HOLDOUT_final")

# Umbral por máximo EV económico (si definiste costos/beneficios)
_ , best_ev = search_best_ev(y_ho, proba_ho)
print(" Tip: ajusta COST_FP, COST_FN, BENEFIT_TP según tu negocio y re-ejecuta.")

# Importancias por permutación (en holdout, usando el pipe base)
print("\n>>> Importancias por permutación (holdout) — top 20")
pi = permutation_importance(pipe, X_ho, y_ho, n_repeats=5, random_state=RANDOM_STATE, scoring='average_precision')
pi_df = pd.DataFrame({"feature": X_ho.columns, "importance_mean": pi.importances_mean, "importance_std": pi.importances_std})
pi_df = pi_df.sort_values("importance_mean", ascending=False)
print(pi_df.head(20).to_string(index=False))
pi_df.to_csv(ARTIF_DIR/"permutation_importance_holdout.csv", index=False)

# (Opcional) SHAP
try:
    import shap
    print("\n>>> Calculando SHAP summary (muestra de holdout).")
    sample_idx = np.random.RandomState(RANDOM_STATE).choice(X_ho.index, size=min(5000, X_ho.shape[0]), replace=False)
    X_sample = X_ho.loc[sample_idx]
    preproc = pipe.named_steps['pre']
    # Importante: ajustar con train_in (no valid/holdout) para no fugar
    _ = preproc.fit(X_tr_in, y_tr_in)
    X_trans = preproc.transform(X_sample)
    model = pipe.named_steps['clf']
    explainer = shap.Explainer(model.predict_proba, X_trans)
    shap_values = explainer(X_trans)
    sv_abs = np.abs(shap_values.values[...,1]).mean(axis=0)
    shap_df = pd.DataFrame({"feature_transformed_id": np.arange(sv_abs.shape[0]), "mean_abs_shap": sv_abs})
    shap_df.to_csv(ARTIF_DIR/"shap_mean_abs_holdout.csv", index=False)
    print("SHAP exportado.")
except Exception as e:
    print("SHAP no disponible/omitido:", e)

# ============================================================
# 7) GUARDAR ARTEFACTOS Y FUNCIÓN DE SCORING
# ============================================================
bundle = {
    "model_calibrado": cal,
    "columns_after_prune": list(Xall.columns),
    "num_final": num_final,
    "cat_final": cat_final,
    "cut1": cut1, "cut2": cut2,
    "experiment_best": best,
    "costs": {"COST_FP":COST_FP,"COST_FN":COST_FN,"BENEFIT_TP":BENEFIT_TP,"COST_TN":COST_TN}
}
joblib.dump(bundle, ARTIF_DIR/"modelo_calibrado.joblib")
pd.DataFrame({"metric":["ROC_AUC","PR_AUC","Brier"],
              "value":[m['roc_auc'], m['pr_auc'], m['brier']]}).to_csv(ARTIF_DIR/"holdout_metrics.csv", index=False)
print(f"\n>>> Artefactos guardados en {ARTIF_DIR.resolve()}")

def score_future(df_nuevo_raw: pd.DataFrame, artif_path=ARTIF_DIR/"modelo_calibrado.joblib",
                 primeruso_strategy=best_exp['primeruso'], ultimo_uso_clip_evento=best_exp['clip_ultimo'],
                 export_csv=True, target_col=TARGET):
    bundle = joblib.load(artif_path)
    model  = bundle["model_calibrado"]
    cols_ok = bundle["columns_after_prune"]
    df_nuevo = parse_and_features(df_nuevo_raw, primeruso_strategy, ultimo_uso_clip_evento)
    Xn = df_nuevo.reindex(columns=cols_ok, fill_value=np.nan)
    proba1 = model.predict_proba(Xn)[:,1]
    proba0 = 1 - proba1
    print("\n[SCORING] Resumen probabilidades nueva data:")
    print(f"mean(p1)= {proba1.mean():.3f} | min/max p1= {proba1.min():.3f}/{proba1.max():.3f}")
    if target_col in df_nuevo.columns:
        y_true = df_nuevo[target_col].astype(int)
        _ = evaluate_proba(y_true, proba1, name="NUEVA_DATA")
    out = pd.DataFrame({"p_no_perdida": proba0, "p_perdida": proba1})
    if export_csv:
        out_path = ARTIF_DIR/"scoring_df_nuevo.csv"
        out.to_csv(out_path, index=False)
        print(f"Scoring exportado a {out_path}")
    return out

# ============================================================
# 8) SNIPPET FASTAPI
# ============================================================
FASTAPI_SNIPPET = f"""
from fastapi import FastAPI
import joblib
import pandas as pd

app = FastAPI(title="API Riesgo Temu", version="1.0")

BUNDLE_PATH = "{str((ARTIF_DIR/'modelo_calibrado.joblib').resolve())}"

from __main__ import parse_and_features  # reutilizamos funciones

bundle = joblib.load(BUNDLE_PATH)
model = bundle["model_calibrado"]
cols_ok = bundle["columns_after_prune"]
best_exp = bundle.get("experiment_best", None)

@app.post("/score")
def score(payload: dict):
    df = pd.DataFrame([payload])
    primeruso = "clip_a_vinc" if not best_exp else best_exp.get("primeruso","clip_a_vinc")
    clip_ult  = True if not best_exp else bool(best_exp.get("clip_ultimo", True))
    df = parse_and_features(df, primeruso_strategy=primeruso, ultimo_uso_clip_evento=clip_ult)
    Xn = df.reindex(columns=cols_ok, fill_value={{}})
    proba1 = model.predict_proba(Xn)[:,1]
    return {{"p_perdida": float(proba1[0])}}
"""
with open(ARTIF_DIR/"app_fastapi_snippet.py", "w") as f:
    f.write(FASTAPI_SNIPPET)
print(f">>> Snippet FastAPI guardado en {ARTIF_DIR/'app_fastapi_snippet.py'}")


>>> Cargando Excel .xlsb ...
Hojas encontradas: ['DataFramePrueba', 'diccionario']

>>> SHAPE crudo: (146939, 30)
>>> Columnas (primeras 30): ['IdentificadorCliente', 'FechaEvento', 'UsabilidadCupo', 'CategoriaPrincipalCredito', 'DiasMaximosMoraCreditosGenerados', 'NumeroCreditosGPrevius', 'NumeroCreditosGCanalFPrevius', 'NumeroCreditosGEstadoActivosPrevius', 'NumeroCreditosGEstadoPagadosPrevius', 'NumeroCreditosGCanalVPrevius', 'NumeroCreditosLPrevius', 'NumeroCreditosLEstadoActivosPrevius', 'NumeroCreditosLEstadoPagadosPrevius', 'FechaVinculacionCliente', 'FechaPrimerUso', 'FechaUltimoUso', 'TotalPagosEfectuadosGlobalmentePrevius', 'TotalPagosEfectuadosLocalmentePrevius', 'CodigoAlmacenEntregaTC', 'CodigoMunicipioEntregaTC', 'TipoMunicipioEntregaTC', 'CanalMunicipioEntregaTC', 'NumeroIntentosFallidos', 'CupoAprobado', 'UsoAppWeb', 'ScoreCrediticio', 'Genero', 'Edad', 'DiasMora', 'PerdidaCartera']

>>> Chequeos de integridad (fechas):
 - PrimerUso < Vinculacion: 20325 (13.8%)
 - Ultim

In [3]:
def resumen_split(df, mask, nombre):
    sub = df.loc[mask, 'FechaEvento_dt']
    print(f"\n[{nombre}] n={mask.sum()}")
    if sub.notna().any():
        print(f"  min={sub.min().date()} | max={sub.max().date()}")
    else:
        print("  (todas las fechas son NaT)")

# 1) Rango por split
resumen_split(clientes, train_in_idx, "train_in (≤ cut1)")
resumen_split(clientes, valid_in_idx, "valid_in (cut1 < · ≤ cut2)")
resumen_split(clientes, holdout_idx,  "holdout  (> cut2)")

# 2) Chequeos de integridad (orden temporal y no solape)
assert set(clientes.index[train_in_idx]).isdisjoint(clientes.index[valid_in_idx])
assert set(clientes.index[train_in_idx]).isdisjoint(clientes.index[holdout_idx])
assert set(clientes.index[valid_in_idx]).isdisjoint(clientes.index[holdout_idx])

max_train = clientes.loc[train_in_idx, 'FechaEvento_dt'].max()
min_valid = clientes.loc[valid_in_idx, 'FechaEvento_dt'].min()
max_valid = clientes.loc[valid_in_idx, 'FechaEvento_dt'].max()
min_hold  = clientes.loc[holdout_idx,  'FechaEvento_dt'].min()

print("\n>>> Verificación de cortes:")
print("  cut1:", pd.Timestamp(cut1).date(), "| cut2:", pd.Timestamp(cut2).date())
print("  max(train) ≤ cut1 ? ", max_train <= cut1)
print("  min(valid)  > cut1 ? ", min_valid  >  cut1)
print("  max(valid) ≤ cut2 ? ", max_valid <= cut2)
print("  min(hold)   > cut2 ? ", min_hold   >  cut2)

# 3) ¿Cubre todo el dataset (excepto NaT)?
mask_any = train_in_idx | valid_in_idx | holdout_idx
n_total = clientes['FechaEvento_dt'].notna().sum()
print("\n>>> Cobertura partición (excluye NaT):")
print(f"  asignadas={mask_any.sum()} de {n_total} con fecha válida")
assert mask_any.sum() == n_total, "Hay filas con fecha válida que no cayeron en ningún split."

# 4) (Opcional) Muestra 3 fechas cercanas al borde para tranquilidad
print("\n>>> Muestras cercanas a cut1/cut2:")
print(clientes[['IdentificadorCliente','FechaEvento_dt']].loc[
    clientes['FechaEvento_dt'].between(cut1 - pd.Timedelta(days=2), cut1 + pd.Timedelta(days=2))
].sort_values('FechaEvento_dt').head(8).to_string(index=False))

print(clientes[['IdentificadorCliente','FechaEvento_dt']].loc[
    clientes['FechaEvento_dt'].between(cut2 - pd.Timedelta(days=2), cut2 + pd.Timedelta(days=2))
].sort_values('FechaEvento_dt').head(8).to_string(index=False))



[train_in (≤ cut1)] n=94041
  min=2022-05-01 | max=2023-05-13

[valid_in (cut1 < · ≤ cut2)] n=23510
  min=2023-05-13 | max=2023-07-29

[holdout  (> cut2)] n=29388
  min=2023-07-29 | max=2023-10-31

>>> Verificación de cortes:
  cut1: 2023-05-13 | cut2: 2023-07-29
  max(train) ≤ cut1 ?  True
  min(valid)  > cut1 ?  True
  max(valid) ≤ cut2 ?  True
  min(hold)   > cut2 ?  True

>>> Cobertura partición (excluye NaT):
  asignadas=146939 de 146939 con fecha válida

>>> Muestras cercanas a cut1/cut2:
 IdentificadorCliente          FechaEvento_dt
                91798 2023-05-11 21:53:02.980
                46661 2023-05-11 21:54:59.957
               125977 2023-05-11 21:55:24.190
               119272 2023-05-11 21:56:03.770
                31315 2023-05-11 21:57:28.290
                51686 2023-05-11 22:01:47.593
               128791 2023-05-11 22:02:56.230
                25380 2023-05-11 22:08:10.203
 IdentificadorCliente          FechaEvento_dt
               100418 2023-07-27 19:56:

-------------

# Reporte Automatizacion

In [7]:
exp_df = pd.DataFrame(exp_results).sort_values(["valid_pr_auc","valid_roc_auc"], ascending=False)
print("\n====================== RESULTADOS DE EXPERIMENTOS (ordenado por PR-AUC VALID) ======================")
print(exp_df.to_string(index=False))



====================== RESULTADOS DE EXPERIMENTOS (ordenado por PR-AUC VALID) ======================
                     exp   primeruso  clip_ultimo  valid_pr_auc  valid_roc_auc  valid_brier  train_pr_auc  train_roc_auc
   mantener__clip_ultimo    mantener         True      0.756547       0.870612     0.117141      0.745563       0.884077
clip_a_vinc__clip_ultimo clip_a_vinc         True      0.755922       0.870066     0.117397      0.747341       0.883777
   poner_na__clip_ultimo    poner_na         True      0.755392       0.870729     0.117097      0.744647       0.883514


In [8]:
# Guardar resumen de experimentos
exp_df.to_csv(ARTIF_DIR/"experiments_summary.csv", index=False)


In [9]:
topk_df = topk_gain_table(y_ho, proba_ho, TOP_K_LIST)
topk_df.to_csv(ARTIF_DIR/"holdout_topk_gain.csv", index=False)



[GANANCIA/LIFT por top-k%]
  top_%  n_alertas  morosos_detectados  tasa_moros_topk  lift_vs_base
     1        293                 211            0.720          3.47
     2        587                 430            0.733          3.53
     5       1469                1099            0.748          3.61
    10       2938                1932            0.658          3.17
    20       5877                3090            0.526          2.53


In [10]:
ev_grid, best_ev = search_best_ev(y_ho, proba_ho)
ev_grid.to_csv(ARTIF_DIR/"holdout_ev_grid.csv", index=False)
pd.DataFrame([best_ev]).to_csv(ARTIF_DIR/"holdout_best_ev.csv", index=False)



[EV] Mejor umbral por valor económico: thr=0.140 | EV=8976.0 | TP=5578.0 FP=10741.0 FN=519.0 TN=12550.0


In [14]:
# dentro de tu venv, si usas uno
!pip install markdown



In [15]:
# ============================================================
# GENERADOR DE INFORME AUTOMÁTICO (Markdown + opcional HTML)
# ============================================================
import os, io, json
from pathlib import Path
import pandas as pd
import numpy as np
import joblib

# Usa el mismo ARTIF_DIR que definiste en el script principal
ARTIF_DIR = Path("./artifacts_modelo")
REPORT_MD  = ARTIF_DIR/"informe_modelo.md"
REPORT_HTML= ARTIF_DIR/"informe_modelo.html"  # opcional (si tienes 'markdown' instalado)

# (Opcional) pega aquí tu texto “notas de correlaciones” para que salga en el informe:
NOTAS_CORRELACIONES = """
## 📌 Correlaciones y hallazgos rápidos

- **DiasMora** correlación alta con *PerdidaCartera* (~0.83). Es variable de **estado actual** → no usar como feature predictiva (evitar fuga).
- **Historial de créditos previos**: activos y pagados muestran señal (p. ej., `NumeroCreditosGEstadoActivosPrevius`, `NumeroCreditosLEstadoActivosPrevius`, `NumeroCreditosLPrevius`…).
- **Antigüedades** (meses desde vinculación / primer uso) aportan señal más estable que las fechas crudas.
- **Demografía y cupo** (`Edad`, `CupoAprobado`, `ScoreCrediticio`): correlación negativa moderada → más edad/cupo/score, menos pérdida.
- **Canal/Tipo municipio**: patrón “Virtual > Físico” en pérdida (≈28% vs 16–20%).
- **Género**: leve diferencia; mantener como categórica.
"""

def _exists(p): 
    try: return Path(p).exists()
    except: return False

def _read_csv_safe(path):
    try:
        if _exists(path):
            return pd.read_csv(path)
    except Exception as e:
        print(f"[WARN] No pude leer {path}: {e}")
    return None

def _fmt_pct(x, digits=1):
    try: return f"{100*float(x):.{digits}f}%"
    except: return "NA"

def _table_md(df, max_rows=20):
    if df is None or len(df)==0:
        return "_(sin datos)_"
    df2 = df.copy()
    if len(df2) > max_rows:
        df2 = df2.head(max_rows)
    try:
        return df2.to_markdown(index=False)
    except Exception:
        # por si no está tabulate, devolvemos CSV como fallback
        return "```\n" + df2.to_csv(index=False) + "```"

def _load_bundle():
    bundle_path = ARTIF_DIR/"modelo_calibrado.joblib"
    if not _exists(bundle_path):
        print(f"[WARN] No existe {bundle_path}")
        return None
    return joblib.load(bundle_path)

def _infer_dates_from_bundle(bundle):
    # spliteo temporal (cut1/cut2) y experimento ganador almacenados en bundle
    cuts = {}
    try:
        cuts["cut1"] = pd.to_datetime(bundle.get("cut1", None))
        cuts["cut2"] = pd.to_datetime(bundle.get("cut2", None))
    except Exception:
        pass
    best_exp = bundle.get("experiment_best", {})
    return cuts, best_exp

def _load_key_artifacts():
    return {
        "metrics": _read_csv_safe(ARTIF_DIR/"holdout_metrics.csv"),
        "thr_sweep": _read_csv_safe(ARTIF_DIR/"holdout_threshold_sweep.csv"),
        "topk": _read_csv_safe(ARTIF_DIR/"holdout_topk_gain.csv"),
        "perm_imp": _read_csv_safe(ARTIF_DIR/"permutation_importance_holdout.csv"),
        "deciles": _read_csv_safe(ARTIF_DIR/"calibracion_deciles_HOLDOUT_final.csv"),
        "exp": _read_csv_safe(ARTIF_DIR/"experiments_summary.csv"),
        "ev_grid": _read_csv_safe(ARTIF_DIR/"holdout_ev_grid.csv"),
        "ev_best": _read_csv_safe(ARTIF_DIR/"holdout_best_ev.csv"),
    }

def generar_informe_markdown(extra_notas=NOTAS_CORRELACIONES, export_html=True):
    ARTIF_DIR.mkdir(parents=True, exist_ok=True)

    bundle = _load_bundle()
    cuts, best_exp = _infer_dates_from_bundle(bundle) if bundle else ({}, {})
    artifacts = _load_key_artifacts()

    # Cabecera
    md = io.StringIO()
    md.write("# Informe Automático — Modelo de Riesgo Temu\n")
    md.write("_Validación temporal estricta (sin fuga)._\n\n")

    # Resumen corto (tarjeta ejecutiva)
    md.write("## 🧾 Resumen ejecutivo\n")
    if artifacts["metrics"] is not None:
        met = artifacts["metrics"].set_index("metric")["value"].to_dict()
        md.write(f"- **ROC-AUC (holdout):** {met.get('ROC_AUC', np.nan):.3f}\n")
        md.write(f"- **PR-AUC (holdout):** {met.get('PR_AUC', np.nan):.3f}\n")
        md.write(f"- **Brier (holdout):** {met.get('Brier', np.nan):.3f}\n")
    if best_exp:
        md.write(f"- **Variante ganadora:** `{best_exp.get('exp','NA')}` | "
                 f"PR-AUC(VALID)={best_exp.get('valid_pr_auc', np.nan):.3f} | "
                 f"ROC-AUC(VALID)={best_exp.get('valid_roc_auc', np.nan):.3f}\n")
    if cuts:
        c1 = cuts.get("cut1", None)
        c2 = cuts.get("cut2", None)
        if c1 is not None and c2 is not None:
            md.write(f"- **Cortes temporales:** train≤**{str(pd.Timestamp(c1).date())}**, "
                     f"valid≤**{str(pd.Timestamp(c2).date())}**, holdout>**{str(pd.Timestamp(c2).date())}**\n")
    md.write("\n")

    # Experimentos
    md.write("## 🔬 Comparativa de experimentos (VALID)\n")
    md.write(_table_md(artifacts["exp"]))
    md.write("\n\n")

    # Métricas holdout (detalle)
    md.write("## 📈 Métricas en HOLDOUT (final)\n")
    md.write(_table_md(artifacts["metrics"]))
    md.write("\n\n")

    # Barrido de umbrales (top 15)
    md.write("## 🎯 Barrido de umbrales (clase=1)\n")
    md.write(_table_md(artifacts["thr_sweep"], max_rows=15))
    md.write("\n\n")

    # Top-k / Lift
    md.write("## 🚀 Ganancia/Lift por Top-k%\n")
    md.write(_table_md(artifacts["topk"]))
    md.write("\n\n")

    # EV económico
    md.write("## 💰 Valor esperado por umbral\n")
    if artifacts["ev_best"] is not None and len(artifacts["ev_best"])>0:
        row = artifacts["ev_best"].iloc[0].to_dict()
        md.write(f"- **Mejor EV:** thr={row.get('thr','NA')} | EV={row.get('EV','NA')} "
                 f"| TP={row.get('TP','NA')} FP={row.get('FP','NA')} FN={row.get('FN','NA')} TN={row.get('TN','NA')}\n\n")
    md.write(_table_md(artifacts["ev_grid"], max_rows=20))
    md.write("\n\n")

    # Calibración por deciles
    md.write("## ⚖️ Calibración por deciles\n")
    md.write(_table_md(artifacts["deciles"]))
    md.write("\n\n")

    # Importancias por permutación (Top-20)
    md.write("## 🔍 Importancias por permutación (Top-20)\n")
    imp = artifacts["perm_imp"]
    if imp is not None and "importance_mean" in imp.columns:
        imp = imp.sort_values("importance_mean", ascending=False)
    md.write(_table_md(imp, max_rows=20))
    md.write("\n\n")

    # Notas (tu texto de correlaciones / hallazgos)
    if extra_notas and extra_notas.strip():
        md.write("## 📝 Notas/lecturas rápidas\n")
        md.write(extra_notas.strip() + "\n\n")

    # Costeo (desde bundle)
    if bundle and "costs" in bundle:
        md.write("## ⚙️ Parámetros de costo/beneficio usados\n")
        md.write("```json\n" + json.dumps(bundle["costs"], indent=2) + "\n```\n\n")

    # Guardar MD
    with open(REPORT_MD, "w", encoding="utf-8") as f:
        f.write(md.getvalue())
    print(f"[OK] Informe Markdown generado en: {REPORT_MD}")

    # (Opcional) Exportar HTML si tienes instalado 'markdown' (pip install markdown)
    if export_html:
        try:
            import markdown as _md
            html = _md.markdown(md.getvalue(), extensions=["tables"])
            with open(REPORT_HTML, "w", encoding="utf-8") as f:
                f.write(f"<!doctype html><meta charset='utf-8'><style>table{{border-collapse:collapse}}td,th{{border:1px solid #ddd;padding:6px}}</style>{html}")
            print(f"[OK] Informe HTML generado en: {REPORT_HTML}")
        except Exception as e:
            print(f"[WARN] No se generó HTML (instala 'markdown' si lo quieres): {e}")

# Si quieres poder ejecutarlo aparte:
if __name__ == "__main__":
    generar_informe_markdown()


[OK] Informe Markdown generado en: artifacts_modelo/informe_modelo.md
[OK] Informe HTML generado en: artifacts_modelo/informe_modelo.html
